In [2]:
###basic csv file creation
import csv

with open('persons.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Name', 'Profession'])
    writer.writerow(['Derek', 'Software Developer'])
    writer.writerow(['Steve', 'Software Developer'])
    writer.writerow(['Paul', 'Manager'])

In [80]:
### writing multiple rows from a python list
import csv
row_list = [["SN", "Name", "Contribution"],
            [1, "Linus Torvalds", "Linux Kernel"],
            [2, "Tim Berners-Lee", "World Wide Web"],
            [3, "Guido van Rossum", "Python Programming"]]
with open('protagonist.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(row_list)

In [3]:
### importing midi
from mido import MidiFile ###this works though
mid = MidiFile('ockIntroitus.mid')

### printing midi data;;; be advised that in MIDI format a note_on messsage with a velocity of 0 is equivalent to a note_off message
for i, track in enumerate(mid.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        print(str(msg))

#how to see how many tracks in a MidiFile
mid.print_tracks(meta_only=True)

# mido.ticks2second(3840,mid.ticks_per_beat,)

Track 0: Missa pro defunctis - Introitus by Johannes Ockeghem
MetaMessage('track_name', name='Missa pro defunctis - Introitus by Johannes Ockeghem', time=0)
MetaMessage('copyright', text='(c) 2001-02 by MAB Soloists', time=0)
MetaMessage('set_tempo', tempo=600000, time=0)
MetaMessage('set_tempo', tempo=300000, time=432)
MetaMessage('set_tempo', tempo=600000, time=14400)
MetaMessage('set_tempo', tempo=300000, time=768)
MetaMessage('set_tempo', tempo=600000, time=9408)
MetaMessage('set_tempo', tempo=300000, time=432)
MetaMessage('end_of_track', time=0)
Track 1: 
program_change channel=0 program=79 time=0
control_change channel=0 control=10 value=96 time=0
control_change channel=0 control=10 value=64 time=0
note_on channel=0 note=77 velocity=100 time=0
note_on channel=0 note=77 velocity=0 time=47
note_on channel=0 note=77 velocity=100 time=1
note_on channel=0 note=77 velocity=0 time=47
note_on channel=0 note=79 velocity=100 time=1
note_on channel=0 note=79 velocity=0 time=47
note_on chann

In [7]:
### Getting only note_on messages
from mido import MidiFile
mid = MidiFile('ockIntroitus.mid')

def test(example):
    for i, track in enumerate(example.tracks):
        print('Track {}: {}'.format(i, track.name))
        for msg in track:
            if msg.type == 'note_on':
                if msg.velocity == 0:
                    print('note_off ' + str(msg)[8:]) # this is for MIDI tracks that implement note_offs as zero velocity note_ons ; standardizes to using note_offs; we will use something like this later
                else: print(str(msg))

In [111]:
### \\\ --- Dynamic NODE creation from a midi file --- /// ###

from mido import MidiFile # sudo pip install mido if you don't have this; targets Python 3.6
import itertools
import csv
mid = MidiFile('ockIntroitus.mid')

def midi2list(midi_file): # gets only the note_on and note_off messages, creates a sublist for each, and breaks their parameters into separate strings
    for i, track in enumerate(midi_file.tracks):
        extraction = [ list(str(msg).split(" ")) for msg in track if msg.type == 'note_on' or msg.type == 'note_off' ]
        yield [ 'Track {} {}'.format(i, track.name), list(extraction) ]

raw_midi_list = list(midi2list(mid))
#print(raw_midi_list)

def get_full(some_list): # filters out empty message lists (usually the meta message list is stored in track 0, so it's empty from the midi2list function filtering only note_on/note_off messages)
    return [ i for i in some_list if len(i[1]) != 0]

full_messages = get_full(raw_midi_list)
# print(full_messages)

def numberer(message_list): # adds number to beginning of each sub-list, which will be used in .csv creation to indicate row number (turns out this is not needed, I misunderstood how csv writer worked...)
    for entry in message_list:
        counter = 1
        for message in entry[1]:
            message.insert(0, counter)
            counter += 1
    return message_list

step3 = numberer(full_messages)
# print(step3)

def standardizer(numbed_list): # tests for note_on messages with velocity = 0; changes them to note_offs
    for entry in numbed_list:
        for message in entry[1]:
            if message[4] == 'velocity=0':
                 message[1] = 'note_off'
    return numbed_list

step4 = standardizer(step3)
# print(step4)

combined_func = standardizer(numberer(get_full(list(midi2list(mid)))))
#print(combined_func)

def abs_onsets(pl): # gets just the onset delta-times and converts them to an int list of absolute times
    for entry in pl:
        deltas = [ int( message[5].partition('=')[2]) for message in entry[1] ]
        yield itertools.accumulate(deltas)

onsetz = list(abs_onsets(combined_func))

def lister(blah): # this is just dealing with the yield iterator
    return [ list(item) for item in blah  ]

list_onsetz = lister(onsetz)
#print(list_onsetz)

def final_format(combined_func, onsets): # finally putting everything together
    for i, entry in enumerate(combined_func):
        for j in range(0,len(entry[1]), 2): #we want to step through 'combined_func' by twos, to get the start and end time of a single node from a note_on message and the note_off immediately following
            onset = onsets[i][j]
            terminus = onsets[i][j+1]
            vertex_id = combined_func[i][1][j][3].partition('=')[2]
            onset_censored = 'FALSE'
            terminus_censored = 'FALSE'
            duration = terminus - onset
            yield [ onset, terminus, vertex_id, onset_censored, terminus_censored, duration ]

node_partition_sizes = [len(entry[1]) // 2 for entry in combined_func] # but the yield doesn't distinguish between tracks, so we have to make this list for how many messages each track contains
#print(node_partition_sizes)

ff = final_format(combined_func, list_onsetz)
# print(list(ff))

partitioned_list = [list(itertools.islice(ff, elem)) for elem in node_partition_sizes] #islice takes a generator for its first argument! i.e. the yield from 'final_format'. And we use 'node_partition_sizes' to tell islice how to group messages from the yield generator.
print(partitioned_list)

def csv_dynamic_node_maker(partitioned_list): #making a .csv file out of each track's messages in 'partitioned_list' and saving it
    for i, entry in enumerate(partitioned_list):
        entry = [['onset', 'terminus', 'vertex.id', 'onset.censored', 'terminus.censored', 'duration']] + entry # column header labels
        title = i + 1 # titling each .csv file by its track's index ( we filtered out MetaMessage track 0, so need to add 1 to index to accurately reflect original track number)
        with open('track_{}_dynamic_nodes.csv'.format(title), 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(entry)

csv_dynamic_node_maker(partitioned_list) #these are the dynamic node lists used in R; saved to same directory this script is in

[[[0, 47, '77', 'FALSE', 'FALSE', 47], [48, 95, '77', 'FALSE', 'FALSE', 47], [96, 143, '79', 'FALSE', 'FALSE', 47], [144, 191, '77', 'FALSE', 'FALSE', 47], [192, 239, '77', 'FALSE', 'FALSE', 47], [239, 430, '65', 'FALSE', 'FALSE', 191], [431, 622, '62', 'FALSE', 'FALSE', 191], [623, 814, '64', 'FALSE', 'FALSE', 191], [815, 1198, '65', 'FALSE', 'FALSE', 383], [1199, 1390, '67', 'FALSE', 'FALSE', 191], [1391, 1582, '69', 'FALSE', 'FALSE', 191], [1583, 1774, '71', 'FALSE', 'FALSE', 191], [1775, 1966, '72', 'FALSE', 'FALSE', 191], [1967, 2158, '65', 'FALSE', 'FALSE', 191], [2159, 2350, '67', 'FALSE', 'FALSE', 191], [2351, 2734, '65', 'FALSE', 'FALSE', 383], [2735, 3118, '65', 'FALSE', 'FALSE', 383], [3119, 3406, '65', 'FALSE', 'FALSE', 287], [3407, 3454, '64', 'FALSE', 'FALSE', 47], [3455, 3502, '62', 'FALSE', 'FALSE', 47], [3503, 3886, '64', 'FALSE', 'FALSE', 383], [3887, 4078, '62', 'FALSE', 'FALSE', 191], [4079, 4462, '65', 'FALSE', 'FALSE', 383], [4559, 4750, '72', 'FALSE', 'FALSE', 19

In [96]:
### Dynamic EDGE creation from Dynamic nodes ###

def edge_converter(partitioned_list):
    for track in partitioned_list:
        for j in range(len(track) - 1): #we need to stop this iterative process before we reach the last message so that j doesn't go out of range
            onset = track[j][1]
            terminus = track[j+1][1]
            tail = track[j][2]
            head = track[j+1][2]
            onset_censored = 'FALSE'
            terminus_censored = 'FALSE'
            duration = terminus - onset
            edge_id = tail + head
            yield [onset, terminus, tail, head, onset_censored, terminus_censored, duration, edge_id ]

edge_conversion = edge_converter(partitioned_list)
# print(list(edge_conversion))

edge_partition_sizes = [i-1 for i in node_partition_sizes]# num(edges) is always num(nodes) - 1, in each track
# print(edge_partition_sizes)

partitioned_edge_conversion = [list(itertools.islice(edge_conversion, elem)) for elem in edge_partition_sizes] # like before the yield generator needs to be told how to group results into tracks
# print(partitioned_edge_conversion)

def edge_id_maker(partitioned_edge_conversion): #generates a list of unordered sets containing the unique node_pair strings for each track
    for track in partitioned_edge_conversion:
        node_pair_set = set()
        for msg in track:
            node_pair_set.add(msg[7])
        yield node_pair_set

edge_list = list(edge_id_maker(partitioned_edge_conversion)) # this is a list of lists of strings (for each track) of the form ''tail'+'head'', i.e. '7172' or '26127' etc.; we assign a unique int to each unique string in the next step
#print(edge_list)

def edge_id_dict_maker(edge_list): # list of dictionaries containing node_pair strings as keys and ints as values
    return [{key:count for count, key in enumerate(edge, 1)} for edge in edge_list]

edge_id_dict_list = edge_id_dict_maker(edge_list)
#print(edge_id_dict_list)

def edge_id_converter(partitioned_edge_conversion, edge_id_dict_list): # replacing each unique node_pair string with its dict value (per track)
    dummy = partitioned_edge_conversion.copy()
    for i, track in enumerate(dummy):
        for msg in track:
            msg.append(edge_id_dict_list[i][msg[7]])
            msg.pop(7)
    return dummy

edge_id_conversion = edge_id_converter(partitioned_edge_conversion, edge_id_dict_list)
print('edge_id_conversion',edge_id_conversion)

def csv_dynamic_edge_maker(edge_id_conversion):
    for i, entry in enumerate(edge_id_conversion):
        entry = [['onset', 'terminus', 'tail', 'head', 'onset.censored', 'terminus.censored', 'duration', 'edge.id']] + entry # column header labels
        title = i + 1 # titling each .csv file by its index ( we filtered out MetaMessage track 0, so need to add 1 to index to accurately reflect original track number)
        with open('track_{}_dynamic_edges.csv'.format(title), 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(entry)

csv_dynamic_edge_maker(edge_id_conversion) # these are the dynamic edge lists, output and saved

def static_edge_maker_conversion(edge_id_conversion): # getting only the tail and head info from the dynamic edge list
    for track in edge_id_conversion:
        yield [[msg[2], msg[3], '1'] for msg in track ] #1 is for adding weight; useful for summing total arrows in R to give aggregate weights

static_edge_list = list(static_edge_maker_conversion(edge_id_conversion))
#print('static_edge_list',static_edge_list)

def csv_static_edge_maker(static_edge_list):
    for i, entry in enumerate(static_edge_list):
        entry = [['tail', 'head', 'weight']] + entry #column header labels
        title = i + 1 # titling each .csv file by its index ( we filtered out MetaMessage track 0, so need to add 1 to index to accurately reflect original track number)
        with open('track_{}_static_edges.csv'.format(title), 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(entry)
            #print(entry)

csv_static_edge_maker(static_edge_list)

edge_id_conversion [[[47, 95, '77', '77', 'FALSE', 'FALSE', 48, 3], [95, 143, '77', '79', 'FALSE', 'FALSE', 48, 27], [143, 191, '79', '77', 'FALSE', 'FALSE', 48, 17], [191, 239, '77', '77', 'FALSE', 'FALSE', 48, 3], [239, 430, '77', '65', 'FALSE', 'FALSE', 191, 14], [430, 622, '65', '62', 'FALSE', 'FALSE', 192, 18], [622, 814, '62', '64', 'FALSE', 'FALSE', 192, 19], [814, 1198, '64', '65', 'FALSE', 'FALSE', 384, 47], [1198, 1390, '65', '67', 'FALSE', 'FALSE', 192, 28], [1390, 1582, '67', '69', 'FALSE', 'FALSE', 192, 4], [1582, 1774, '69', '71', 'FALSE', 'FALSE', 192, 32], [1774, 1966, '71', '72', 'FALSE', 'FALSE', 192, 44], [1966, 2158, '72', '65', 'FALSE', 'FALSE', 192, 38], [2158, 2350, '65', '67', 'FALSE', 'FALSE', 192, 28], [2350, 2734, '67', '65', 'FALSE', 'FALSE', 384, 1], [2734, 3118, '65', '65', 'FALSE', 'FALSE', 384, 15], [3118, 3406, '65', '65', 'FALSE', 'FALSE', 288, 15], [3406, 3454, '65', '64', 'FALSE', 'FALSE', 48, 16], [3454, 3502, '64', '62', 'FALSE', 'FALSE', 48, 2], [

In [46]:
### Vertex attributes sub-routine ###
# two ways to do this

chromatic_pitches = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']

def MIDI_pitch_mapping(chromatic_pitches):
    i = 0
    for j in range(24,128):
        note = chromatic_pitches[(j-24) % 12]
        if note == 'C':
            i += 1
        yield [ j, note + str(i) ]

shame = [[21, 'A0'], [22, 'A#0'], [23, 'B0']] #lol ugly hack, hence 'shame'

MIDI_map = shame + list(MIDI_pitch_mapping(chromatic_pitches))
#print('MIDI_map',MIDI_map)

def csv_total_vertex_attributes_maker(MIDI_map): # first way is to include every possible MIDI note name in the vertex attributes file
    alias = MIDI_map[:]
    alias.insert(0, ['vertex.id', 'note_name',]) # column header labels
    with open('MIDI_map.csv', 'w', newline='') as file: 
        writer = csv.writer(file)
        writer.writerows(alias)

csv_total_vertex_attributes_maker(MIDI_map) # output and saved; this should be sent to R SNA package as vertex attributes, aka vertex.attr

### Second Way ###
def get_node_sets(partitioned_list): # getting an unordered set of nodes present in each track; we will need this if we want to send a vertexAttributes file to R with ONLY those notes used in a track as vertices, instead of full MIDI_map
    for track in partitioned_list:
        node_set = set()
        for msg in track:
            node_set.add(int(msg[2]))
        yield node_set

node_sets = list(get_node_sets(partitioned_list))
#print('node_sets',node_sets)

node_lists = [ sorted(list(node_set)) for node_set in node_sets ] #ordering each set
#print('node_lists',node_lists)

node_list_partition_sizes = [ len(track) for track in node_lists ] #same method from before; we're going to need this for our number_from_1 function to tell yield how to group tracks using islice
#print('node_list_partition_sizes',node_list_partition_sizes)

def node_filter(MIDI_map): #structures node_list with midi_key and note_name string, to be used in .csv creator
            return [ list(filter(lambda m: m[0] in node_list, MIDI_map)) for node_list in node_lists ] # filter generators have no intrinsic type so type needs to be specificied by running list() inside comprehension

formatted_node_lists = node_filter(MIDI_map)
print('formatted_node_lists',list(formatted_node_lists))

def number_from_1(filt_list):
    for track in (filt_list):
        for countish, msg in enumerate(track):
             yield [ countish + 1, msg[1] ]

proper_numbered = number_from_1(formatted_node_lists)

grouping_proper_numbered = [list(itertools.islice(proper_numbered, elem)) for elem in node_list_partition_sizes]
print('grouping_proper_numbered',grouping_proper_numbered)

def csv_vertex_attr_by_track(grouping_proper_numbered):
    for i, entry in enumerate(grouping_proper_numbered):
        entry = [['vertex.id', 'name']] + entry #column header labels
        title = i + 1 # titling each .csv file by its index ( we filtered out MetaMessage track 0, so need to add 1 to index to accurately reflect original track number)
        with open('track_{}_static_nodes.csv'.format(title), 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(entry)

csv_vertex_attr_by_track(grouping_proper_numbered)

formatted_node_lists [[[60, 'C4'], [62, 'D4'], [64, 'E4'], [65, 'F4'], [67, 'G4'], [69, 'A4'], [70, 'A#4'], [71, 'B4'], [72, 'C5'], [74, 'D5'], [77, 'F5'], [79, 'G5'], [81, 'A5']], [[58, 'A#3'], [60, 'C4'], [62, 'D4'], [64, 'E4'], [65, 'F4'], [67, 'G4'], [69, 'A4'], [70, 'A#4'], [71, 'B4'], [72, 'C5'], [74, 'D5'], [76, 'E5'], [77, 'F5'], [79, 'G5']], [[72, 'C5'], [74, 'D5'], [76, 'E5'], [77, 'F5'], [79, 'G5'], [81, 'A5'], [82, 'A#5'], [83, 'B5'], [84, 'C6']]]
grouping_proper_numbered [[[1, 'C4'], [2, 'D4'], [3, 'E4'], [4, 'F4'], [5, 'G4'], [6, 'A4'], [7, 'A#4'], [8, 'B4'], [9, 'C5'], [10, 'D5'], [11, 'F5'], [12, 'G5'], [13, 'A5']], [[1, 'A#3'], [2, 'C4'], [3, 'D4'], [4, 'E4'], [5, 'F4'], [6, 'G4'], [7, 'A4'], [8, 'A#4'], [9, 'B4'], [10, 'C5'], [11, 'D5'], [12, 'E5'], [13, 'F5'], [14, 'G5']], [[1, 'C5'], [2, 'D5'], [3, 'E5'], [4, 'F5'], [5, 'G5'], [6, 'A5'], [7, 'A#5'], [8, 'B5'], [9, 'C6']]]


In [108]:
#Various hacks to fix issue of R assuming I want all nodes drawn up to a certain vertex.id;
#  so if I just use MIDI numbers as vertex ids, then if the highest midi number used is, for example say, 72, then it will draw 72 nodes, even if vertex.ids 1-60 are never used.

master=[]
for track in grouping_proper_numbered:
    new_track=[]
    for msg in track:
        new_track.append(msg[0])
    master.append(new_track)
print('master',master)

final=[]
for track in formatted_node_lists:
    new_track=[]
    for msg in track:
        new_track.append(msg[0])
    final.append(new_track)

node_id_dict=[]
for i in range(len(master)):
    node_id_dict.append(dict(list(zip(final[i],master[i]))))
print('horizon',node_id_dict)

ghost=[]
for i, track in enumerate(static_edge_list):
    new_track=[]
    for msg in track:
        new_track.append([node_id_dict[i][int(msg[0])],node_id_dict[i][int(msg[1])],msg[2]])
    ghost.append(new_track)
#print('ghost',ghost)
csv_static_edge_maker(ghost)

spirit=[]
for i, track in enumerate(edge_id_conversion):
    new_track=[]
    for msg in track:
        new_track.append([msg[0],msg[1],node_id_dict[i][int(msg[2])],node_id_dict[i][int(msg[3])],msg[4],msg[5],msg[6],msg[7]])
    spirit.append(new_track)
#print('spirit',spirit)
csv_dynamic_edge_maker(spirit)

meme=[]
for i, track in enumerate(partitioned_list):
    new_track=[]
    for msg in track:
        new_track.append([msg[0],msg[1],node_id_dict[i][int(msg[2])],msg[3],msg[4],msg[5]])
    meme.append(new_track)
#print('meme',meme)
csv_dynamic_node_maker(meme)


master [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], [1, 2, 3, 4, 5, 6, 7, 8, 9]]
horizon [{60: 1, 62: 2, 64: 3, 65: 4, 67: 5, 69: 6, 70: 7, 71: 8, 72: 9, 74: 10, 77: 11, 79: 12, 81: 13}, {58: 1, 60: 2, 62: 3, 64: 4, 65: 5, 67: 6, 69: 7, 70: 8, 71: 9, 72: 10, 74: 11, 76: 12, 77: 13, 79: 14}, {72: 1, 74: 2, 76: 3, 77: 4, 79: 5, 81: 6, 82: 7, 83: 8, 84: 9}]
